### GPU Usage in Julia

Some simple examples to show how easy it is to do computations on your GPU from Julia. These examples all require an NVIDIA GPU with CUDA installed. There is support also for non-NVIDIA GPUs via OpenCL using the [CLArrays Package](https://github.com/JuliaGPU/CLArrays.jl), though I haven't tested it out.

The packages used here are:

[CUArrays](https://github.com/JuliaGPU/CUArrays.jl): Provides GPU arrays and supports standard operations on them - see the README for more details. Simple to use, no prior knowledge of using, or programming GPUs is really needed.

[CUDAnative](https://github.com/JuliaGPU/CUArrays.jl): Allows you to write your own custom GPU kernels (programs that run in parallel on the GPU) using Julia. Further reading on the advantages of using Julia for this: http://mikeinnes.github.io/2017/08/24/cudanative.html

In [ ]:
# Uncomment the below to install CuArrays

# using Pkg; Pkg.add("CuArrays")

In [1]:
using CuArrays
using BenchmarkTools

Multiply two 1000x1000 element matrices on the cpu

In [2]:
# We use const here because [non-const globals are slow](https://docs.julialang.org/en/v1/manual/performance-tips/index.html#Avoid-global-variables-1)
const dim = 1000
const ac = rand(dim, dim)
const bc = rand(dim, dim)

@btime ac*bc

  20.277 ms (2 allocations: 7.63 MiB)


1000×1000 Array{Float64,2}:
 256.592  248.702  245.313  244.702  …  249.644  245.923  243.769  253.103
 261.498  257.975  253.463  255.377     254.744  254.901  248.419  262.017
 253.64   242.134  241.193  249.984     248.066  240.754  244.385  252.285
 256.114  245.055  244.443  249.676     254.11   244.597  239.771  254.641
 259.852  249.964  249.099  254.542     250.815  246.535  247.636  257.095
 252.457  246.472  243.208  247.211  …  248.711  242.678  247.677  254.618
 255.461  242.733  241.797  252.106     245.892  244.96   249.522  257.296
 254.716  248.481  251.584  251.931     248.815  244.849  247.925  253.883
 266.108  256.503  253.956  252.816     255.772  250.646  253.961  261.284
 249.175  244.057  237.917  250.194     242.806  238.042  241.543  246.982
 250.288  248.48   241.844  249.041  …  246.038  243.617  240.9    253.655
 252.601  249.351  253.049  253.525     247.938  244.167  247.965  253.656
 260.055  254.522  252.469  252.417     259.704  254.414  247.356  262.0

And now on the GPU

In [3]:
const ag = cu(ac)
const bg = cu(bc)
@btime ag*bg

  10.225 μs (11 allocations: 400 bytes)


1000×1000 CuArray{Float32,2}:
 256.592  248.702  245.313  244.702  …  249.644  245.923  243.769  253.103
 261.498  257.975  253.463  255.377     254.744  254.901  248.419  262.017
 253.64   242.134  241.193  249.984     248.066  240.754  244.385  252.285
 256.114  245.055  244.443  249.676     254.11   244.597  239.771  254.641
 259.852  249.964  249.099  254.542     250.815  246.535  247.636  257.096
 252.458  246.472  243.209  247.211  …  248.711  242.678  247.677  254.618
 255.461  242.733  241.797  252.106     245.892  244.96   249.522  257.296
 254.716  248.481  251.584  251.931     248.815  244.848  247.925  253.883
 266.108  256.503  253.956  252.816     255.772  250.646  253.961  261.284
 249.175  244.057  237.917  250.194     242.806  238.042  241.543  246.981
 250.288  248.48   241.843  249.041  …  246.039  243.617  240.9    253.655
 252.601  249.351  253.049  253.524     247.938  244.167  247.965  253.656
 260.055  254.522  252.469  252.417     259.704  254.413  247.356  262

---
Write our own kernel to add two matrices together in parallel on the GPU

In [4]:
# Uncomment the below to install CUDAnative.jl

# using Pkg; Pkg.add("CUDAnative")

Create a known test array so we can eyeball our results to make sure they look correct

In [5]:
function testarr(shape...)
    len = prod(shape) # length (number of elements) is the product of the dimensions
    return collect(reshape(1:len, shape)) # reshape it to "shape"
end

testarr (generic function with 1 method)

In [6]:
# Create two NxN arrays
const N = 1024
const xc = testarr(N, N)
const yc = testarr(N, N)
const zc = zeros(Int64, size(xc)); # an array to store the result

In [7]:
zc .= xc + yc # non-allocating elementwise sum
@btime zc .= xc + yc

  2.653 ms (2 allocations: 8.00 MiB)


1024×1024 Array{Int64,2}:
    2  2050  4098  6146   8194  10242  …  2088962  2091010  2093058  2095106
    4  2052  4100  6148   8196  10244     2088964  2091012  2093060  2095108
    6  2054  4102  6150   8198  10246     2088966  2091014  2093062  2095110
    8  2056  4104  6152   8200  10248     2088968  2091016  2093064  2095112
   10  2058  4106  6154   8202  10250     2088970  2091018  2093066  2095114
   12  2060  4108  6156   8204  10252  …  2088972  2091020  2093068  2095116
   14  2062  4110  6158   8206  10254     2088974  2091022  2093070  2095118
   16  2064  4112  6160   8208  10256     2088976  2091024  2093072  2095120
   18  2066  4114  6162   8210  10258     2088978  2091026  2093074  2095122
   20  2068  4116  6164   8212  10260     2088980  2091028  2093076  2095124
   22  2070  4118  6166   8214  10262  …  2088982  2091030  2093078  2095126
   24  2072  4120  6168   8216  10264     2088984  2091032  2093080  2095128
   26  2074  4122  6170   8218  10266     2088986 

In [8]:
using CuArrays, CUDAnative

xg, yg = CuArray(xc), CuArray(yc)
zg = CuArray(zeros(Int64, size(zc))) # an array to store the result

"""
A very simple kernel to add two arrays
"""
function kernel_vadd(out, a, b)
    # get the index into the arrays that this GPU processor will work on
    i = (blockIdx().x-1) * blockDim().x + threadIdx().x
    
    # add the two elements and store the result
    out[i] = a[i] + b[i]
    
    # kernels must return `nothing`
    return
end

# launch (i.e. run) the GPU kernel with the same number of (threads, blocks) as (rows, columns) in our arrays
@btime @cuda threads=N blocks=N kernel_vadd(zg, xg, yg)

  6.883 μs (33 allocations: 1.23 KiB)


In [9]:
zg

1024×1024 CuArray{Int64,2}:
    2  2050  4098  6146   8194  10242  …  2088962  2091010  2093058  2095106
    4  2052  4100  6148   8196  10244     2088964  2091012  2093060  2095108
    6  2054  4102  6150   8198  10246     2088966  2091014  2093062  2095110
    8  2056  4104  6152   8200  10248     2088968  2091016  2093064  2095112
   10  2058  4106  6154   8202  10250     2088970  2091018  2093066  2095114
   12  2060  4108  6156   8204  10252  …  2088972  2091020  2093068  2095116
   14  2062  4110  6158   8206  10254     2088974  2091022  2093070  2095118
   16  2064  4112  6160   8208  10256     2088976  2091024  2093072  2095120
   18  2066  4114  6162   8210  10258     2088978  2091026  2093074  2095122
   20  2068  4116  6164   8212  10260     2088980  2091028  2093076  2095124
   22  2070  4118  6166   8214  10262  …  2088982  2091030  2093078  2095126
   24  2072  4120  6168   8216  10264     2088984  2091032  2093080  2095128
   26  2074  4122  6170   8218  10266     208898

In [10]:
if zc == zg
    @info "arrays are equal"
else
    @warn "something went wrong: arrays are NOT equal"
end

┌ Info: arrays are equal
└ @ Main In[10]:2
